In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("numbers.csv")
data.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [3]:
X = data.iloc[:, :4].values
Y = data.iloc[:, 4].values.reshape(-1,1)

In [4]:
print(X)
print(Y)

print(np.shape(X))
print(np.shape(Y))

[[  14.96   41.76 1024.07   73.17]
 [  25.18   62.96 1020.04   59.08]
 [   5.11   39.4  1012.16   92.14]
 ...
 [  31.32   74.33 1012.92   36.48]
 [  24.48   69.45 1013.86   62.39]
 [  21.6    62.52 1017.23   67.87]]
[[463.26]
 [444.37]
 [488.56]
 ...
 [429.57]
 [435.74]
 [453.28]]
(9568, 4)
(9568, 1)


In [5]:
from sklearn.model_selection import train_test_split
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, random_state=2021)


Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain, Ytrain, test_size=0.2, random_state=0)

In [6]:
#from sklearn.linear_model import LinearRegression

from sklearn.ensemble import GradientBoostingRegressor
# Create and fit the linear regression model
regr =  GradientBoostingRegressor()
regr.fit(Xtrain, Ytrain)

# Make predictions on the validation set
pred = regr.predict(Xval)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
# make a dictionary of hyperparameter values to search
search_space = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

In [8]:
from sklearn.model_selection import GridSearchCV
# make a GridSearchCV object

GS = GridSearchCV(estimator =  GradientBoostingRegressor(),
                  param_grid = search_space,
                  scoring = ["r2", "neg_mean_squared_error"], #sklearn.metrics.SCORERS.keys()
                  refit = "r2",
                  cv = 5,
                  verbose = 4)

In [9]:
GS.fit(Xtest, Ytest)

TypeError: Parameter grid for parameter 'n_estimators' needs to be a list or a numpy array, but got 500 (of type int) instead. Single values need to be wrapped in a list with one element.

In [ ]:
print("best model: ", GS.best_estimator_) # to get the complete details of the best model
print("best parameter values: ", GS.best_params_) # to get only the best hyperparameter values that we searched for
print("best r^2 value: ", GS.best_score_) # score according to the metric we passed in refit

best model:  LinearRegression(n_jobs=-1)
best parameter values:  {'copy_X': True, 'fit_intercept': True, 'n_jobs': -1, 'positive': False}
best r^2 value:  0.9317961881853025


In [ ]:
df = pd.DataFrame(GS.cv_results_)
df = df.sort_values("rank_test_r2")
df.to_csv("cv_results.csv", index = False)